In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = Path('data/mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set. Since data is different in both datasets and both normalized in same way
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [0]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [0]:
# we are initializing two layers one with nh hidden and last layer we use MSE instead on Crossentropy for understanding.

# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [14]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
def lin(x, w, b): 
    return x@w + b

In [0]:
t = lin(x_valid, w1, b1)

In [17]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0310), tensor(0.9901))

In [0]:
def relu(x): 
    return x.clamp_min(0.) # clamp_min replace negatives with zero's.

In [0]:
# first layer
t = relu(lin(x_valid, w1, b1))

In [20]:
#...actually it really should be this!
t.mean(),t.std()

# we don't have mean near to 0 and std near to 1 as we used Relu and all data below zero are zero now and so SD and mean increased.


(tensor(0.4061), tensor(0.5970))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [22]:
w1.mean(),w1.std()

(tensor(0.0004), tensor(0.0504))

In [23]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4707), tensor(0.7314))

In [0]:
#export
from torch.nn import init

In [0]:
# same as 20 with pytorch method
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # given fanout as we divided by 'm' in math.sqrt(2/m) instead of 'nh'
t = relu(lin(x_valid, w1, b1))

In [26]:
init.kaiming_normal_??
'''
Choosing `fan_in` preserves the magnitude of the variance of the weights in the forward pass. 
Choosing `fan_out` preserves the magnitudes in the backwards pass.
'''

'\nChoosing `fan_in` preserves the magnitude of the variance of the weights in the forward pass. \nChoosing `fan_out` preserves the magnitudes in the backwards pass.\n'

In [27]:
w1.mean(),w1.std()

(tensor(-2.5986e-05), tensor(0.0506))

In [28]:
# almost same as 22
t.mean(),t.std()

(tensor(0.5742), tensor(0.8219))

In [29]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [31]:
torch.nn.Linear(m,nh).weight.shape

# pytorch linear model creates weight shape of 50x784 instead 784x50 if we do manually.

torch.Size([50, 784])

In [0]:
torch.nn.Linear.forward??

In [0]:
torch.nn.functional.linear??

In [0]:
torch.nn.Conv2d??

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
# what if...?
def relu(x): 
    return x.clamp_min(0.) - 0.5 # doing this mean will be near to zero

In [37]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1159), tensor(0.8465))

### forward pass

In [0]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [39]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 23.3 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [41]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [45]:
preds.shape

torch.Size([50000, 1])

In [46]:
mse(preds, y_train)

tensor(33.5028)

### Gradients and backward pass

In [0]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [0]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [0]:
class Relu():
    def __call__(self, inp):  
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): 
        self.inp.g = (self.inp>0).float() * self.out.g

__call__ is our forward pass for the layers.

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [64]:
%time loss = model(x_train, y_train)

CPU times: user 138 ms, sys: 444 µs, total: 138 ms
Wall time: 140 ms


In [65]:
%time model.backward()

CPU times: user 4.04 s, sys: 4.03 s, total: 8.06 s
Wall time: 8.09 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [73]:
%time loss = model(x_train, y_train)

CPU times: user 137 ms, sys: 1.9 ms, total: 139 ms
Wall time: 149 ms


In [74]:
%time model.backward()

CPU times: user 264 ms, sys: 19.4 ms, total: 284 ms
Wall time: 297 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g    # replaced einsum with pytorch's matmul funtion
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [78]:
%time loss = model(x_train, y_train)

CPU times: user 131 ms, sys: 948 µs, total: 132 ms
Wall time: 131 ms


In [79]:
%time model.backward()

CPU times: user 287 ms, sys: 3.76 ms, total: 291 ms
Wall time: 290 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [84]:
%time loss = model(x_train, y_train)

CPU times: user 118 ms, sys: 1.61 ms, total: 120 ms
Wall time: 127 ms


In [85]:
%time loss.backward()

CPU times: user 92 ms, sys: 922 µs, total: 92.9 ms
Wall time: 96 ms


## Export

In [92]:
!chmod 755 ./notebook2script.py
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
